In [1]:
import boto3
import random

s3client = boto3.client('s3')

In [7]:
# Bucket names are globally unique. Here we'll add a random element to the name to increase the odds of
# avoiding a name collision
bucket_name = 'ds-lifecycle-' + str(random.randint(1, 9999999))
print bucket_name

ds-lifecycle-4334435


In [8]:
response = s3client.create_bucket(
    Bucket=bucket_name
)

print response

{u'Location': '/ds-lifecycle-4334435', 'ResponseMetadata': {'HTTPStatusCode': 200, 'RetryAttempts': 0, 'HostId': 'Ilb3YuavEWsJSpoMwzDFhiLrXhciSEfuhqP30wQPqHDiW0iYdZ5qN1Msi4qid8G/nA0DS63ZADw=', 'RequestId': '71DD642BD2646FFF', 'HTTPHeaders': {'content-length': '0', 'x-amz-id-2': 'Ilb3YuavEWsJSpoMwzDFhiLrXhciSEfuhqP30wQPqHDiW0iYdZ5qN1Msi4qid8G/nA0DS63ZADw=', 'server': 'AmazonS3', 'x-amz-request-id': '71DD642BD2646FFF', 'location': '/ds-lifecycle-4334435', 'date': 'Fri, 12 May 2017 12:17:16 GMT'}}}


In [10]:
# Execute this cell to enable versioning. Once versioning is enable for a bucket it cannot be removed,
# although it can be disabled.
response = s3client.put_bucket_versioning(
    Bucket=bucket_name,
    VersioningConfiguration={
        'MFADelete': 'Disabled',
        'Status': 'Enabled'
    }
)

print response

{'ResponseMetadata': {'HTTPStatusCode': 200, 'RetryAttempts': 0, 'HostId': 'akR5bKyy5fRpAhHohQEtEra438e6zNF3TAcwhCPhbw7fm/7e+NyDMvSumsZe+o7YRSzHMtmwa9A=', 'RequestId': '7E875F205C45339D', 'HTTPHeaders': {'x-amz-id-2': 'akR5bKyy5fRpAhHohQEtEra438e6zNF3TAcwhCPhbw7fm/7e+NyDMvSumsZe+o7YRSzHMtmwa9A=', 'date': 'Fri, 12 May 2017 12:17:42 GMT', 'content-length': '0', 'x-amz-request-id': '7E875F205C45339D', 'server': 'AmazonS3'}}}


In [12]:
# Transition the current version of the object to IA storage 30 days after create
response = s3client.put_bucket_lifecycle(
    Bucket=bucket_name,
    LifecycleConfiguration={
        'Rules': [
            {
                'ID' : 'move to IA after 30 rule',
                'Prefix':'',
                'Status':'Enabled',
                'Transition': {
                    'Days': 30,
                    'StorageClass':'STANDARD_IA'
                }
            }
        ]
    }
)

print response

{'ResponseMetadata': {'HTTPStatusCode': 200, 'RetryAttempts': 0, 'HostId': 'DzAIDqfPJBaWvdFXPvffMyXoavzosGHwnnVf2fAPiH7JUh+P8IBJ3po8XOdk1iE6v0NMXhdO5dc=', 'RequestId': 'E7ADB6D8984D5C2A', 'HTTPHeaders': {'x-amz-id-2': 'DzAIDqfPJBaWvdFXPvffMyXoavzosGHwnnVf2fAPiH7JUh+P8IBJ3po8XOdk1iE6v0NMXhdO5dc=', 'date': 'Fri, 12 May 2017 12:20:01 GMT', 'content-length': '0', 'x-amz-request-id': 'E7ADB6D8984D5C2A', 'server': 'AmazonS3'}}}


In [13]:
response = s3client.get_bucket_lifecycle(
    Bucket=bucket_name
)

print response

{u'Rules': [{u'Status': 'Enabled', u'Prefix': '', u'Transition': {u'Days': 30, u'StorageClass': 'STANDARD_IA'}, u'ID': 'move to IA after 30 rule'}], 'ResponseMetadata': {'HTTPStatusCode': 200, 'RetryAttempts': 0, 'HostId': 'GM1Tq32760ec7u5suDwf5+V+DNyvekmfhJ84cgZeR2SUHjb9NzWqW7YyUaAOvI5ECPaHmEEBrgs=', 'RequestId': '082553EB0806E042', 'HTTPHeaders': {'x-amz-id-2': 'GM1Tq32760ec7u5suDwf5+V+DNyvekmfhJ84cgZeR2SUHjb9NzWqW7YyUaAOvI5ECPaHmEEBrgs=', 'date': 'Fri, 12 May 2017 12:20:11 GMT', 'transfer-encoding': 'chunked', 'x-amz-request-id': '082553EB0806E042', 'server': 'AmazonS3'}}}


In [ ]:
f = open("./hello.txt","w")
f.write("hello, world")
f.close()

In [ ]:
s3 = boto3.resource('s3')
response = s3.meta.client.upload_file('./hello.txt', bucket_name, 'hello.txt')
print response

In [ ]:
s3 = boto3.resource('s3')
object_acl = s3.ObjectAcl(bucket_name,'hello.txt')
response = object_acl.put(
    ACL='public-read'
)

print response

In [ ]:
f = open("./hello.txt","w")
f.write("hello, world -- again")
f.close()

In [ ]:
s3 = boto3.resource('s3')
response = s3.meta.client.upload_file('./hello.txt', bucket_name, 'hello.txt')
print response

In [ ]:
s3 = boto3.resource('s3')
object_acl = s3.ObjectAcl(bucket_name,'hello.txt')
response = object_acl.put(
    ACL='public-read'
)

print response

In [ ]:
response = s3client.list_object_versions(
    Bucket=bucket_name,
    Prefix='hello.txt',
    MaxKeys=100
)


versions = response['Versions']
for version in versions:
    print "key: '{}' version {}".format(version['Key'], version['VersionId'])


In [ ]:
# This one does a delete on the object. In the console you can see the deleted versions and marker
response = s3client.delete_object(
        Bucket=bucket_name,
        Key='hello.txt'
)

print response

In [ ]:
response = s3client.list_object_versions(
    Bucket=bucket_name,
    Prefix='hello.txt',
    MaxKeys=100
)


print response

if 'Versions' in response:
    versions = response['Versions']
    for v in versions:
        print v['Key']
else:
    print 'No versions'
    


In [ ]:
if 'DeleteMarkers' in response:
    deleteMarkers = response['DeleteMarkers']
    for dm in deleteMarkers:
        print dm
else:
    print 'No delete markers'

In [ ]:
# This cell deletes all the object versions in the bucket. There's no restore/delete marker after this.
for version in versions:
    response = s3client.delete_object(
        Bucket=bucket_name,
        Key=version['Key'],
        VersionId=version['VersionId']
    )

    print response

In [ ]:
# This cell deletes all the delete markers in the bucket. If you delete an object (just the object, not all the 
# object versions( then delete the deletemarker, this 'restores' the object.
response = s3client.list_object_versions(
    Bucket=bucket_name,
    Prefix='hello.txt',
    MaxKeys=100
)

if 'DeleteMarkers' in response:
    deleteMarkers = response['DeleteMarkers']
    for dm in deleteMarkers:
        response = s3client.delete_object(
            Bucket=bucket_name,
            Key=dm['Key'],
            VersionId=dm['VersionId']
        )
        
        print response
else:
    print 'No delete markers to delete'

In [14]:
response = s3client.delete_bucket(
    Bucket=bucket_name
)


print response

{'ResponseMetadata': {'HTTPStatusCode': 204, 'RetryAttempts': 0, 'HostId': 'jcB+G9UhAGmP5t6LziwEStEYe40rewFoUJlrGSXdOZl31bY/vERbpwe0MKeEPZbFdRE1urRN+SY=', 'RequestId': '4400F02C8B901980', 'HTTPHeaders': {'x-amz-id-2': 'jcB+G9UhAGmP5t6LziwEStEYe40rewFoUJlrGSXdOZl31bY/vERbpwe0MKeEPZbFdRE1urRN+SY=', 'date': 'Fri, 12 May 2017 12:20:37 GMT', 'x-amz-request-id': '4400F02C8B901980', 'server': 'AmazonS3'}}}
